<a href="https://colab.research.google.com/github/maxmonteiro2008/Academic/blob/main/Dasa_Prestadores_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -r mx_requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 18.8 MB/s eta 0:00:00
Processing /colabtools/dist/google-colab-1.0.0.tar.gz (from -r mx_requirements.txt (line 137))
ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/colabtools/dist/google-colab-1.0.0.tar.gz'



In [5]:
import pandas as pd
import numpy as np
import sys

# bibliotecas de análise de perfomance do modelo
import matplotlib.pyplot as plt
from sklearn.tree import export_graphviz
import pydot
from sklearn.metrics import RocCurveDisplay

#bibliotecas de  similaridade de strings
from strsimpy.jaro_winkler import JaroWinkler
from thefuzz import fuzz

#bibliotecas de geração do modelo
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import joblib




In [17]:
# len (sys.argv<4)
if (False):
   print("O número de  parâmetros está incompleto \n")
else:
   #data_arq=sys.argv[1]
   data_arq="./max_dados_4.xlsx"
   model_arq="./random_forest_m1.joblib"
   #model_arq= sys.argv[2]
   #out_arq= sys.argv[3]
   out_arq="./out.xlsx"
   #if(len(data_arq) !=0 ):
   if(True):
      data = pd.read_excel(data_arq, header=0)

   else:
      print("nome do arquivo de entrada, não encontrado")

   if(len(model_arq) !=0 ):
      rf_loaded_model = joblib.load(model_arq)

   else:
      print("nome do arquivo de modelo, não encontrado")
   if(len(out_arq) ==0 ):
      print("nome do arquivo de modsaida, não encontrado")


# limpando duplicados
data.drop_duplicates(inplace = True)
origi=data.copy()


# calculando  Similaridades
distance=JaroWinkler()
for x in data.index:
   # pyjarowinkler similarities razao social Ouro

   if (type(data.loc[x,"razaosocial_base_ouro"])==str) and (type(data.loc[x,["nmPrestador_clean"]])==str):
       data.loc[x,["sim_nmPrestadorCleanRazaosocialOuroJaro"]]=distance.similarity(data.loc[x,"razaosocial_base_ouro"].strip().lower(), data.loc[x,"nmPrestador_clean"].strip().lower())

   else:
       data.loc[x,["sim_nmPrestadorCleanRazaosocialOuroJaro"]]=distance.similarity(data.loc[x,"razaosocial_base_ouro"], data.loc[x,"nmPrestador_clean"])


   if (type(data.loc[x,'razaosocial_base_ouro'])==str)and (type(data.loc[x,"nmFantasia_clean"])==str):
       data.loc[x,["sim_nmFantasiaCleanRazaosocialOuroJaro"]]=distance.similarity(data.loc[x,'razaosocial_base_ouro'].strip().lower(), data.loc[x,"nmFantasia_clean"].strip().lower())
   else:
       data.loc[x,["sim_nmFantasiaCleanRazaosocialOuroJaro"]]=distance.similarity(data.loc[x,'razaosocial_base_ouro'], data.loc[x,"nmFantasia_clean"])


   # fuzz similarities nmFantasia Ouro
   if (type(data.loc[x,"nmPrestador_clean"])==str)and (type(data.loc[x,"nomefantasia_base_ouro"])==str)and (type(data.loc[x,["nmFantasia_clean"]])==str)and (type (data.loc[x,["zaosocial"]])==str):
      data.loc[x,["sim_nmPrestadorCleannmFantasiaOuroFuzz"]]=fuzz.ratio(data.loc[x,"nmPrestador_clean"].strip().lower(), data.loc[x,'nomefantasia_base_ouro'].strip().lower())/100
      data.loc[x,["sim_nmFantasiaCleannmFantasiaOuroFuzz"]]=fuzz.ratio(data.loc[x,"nmFantasia_clean"].strip().lower(), data.loc[x,'nomefantasia_base_ouro'].strip().lower())/100


   else:

      data.loc[x,["sim_nmPrestadorCleannmFantasiaOuroFuzz"]]=fuzz.ratio(data.loc[x,"nmPrestador_clean"], data.loc[x,'nomefantasia_base_ouro'])/100
      data.loc[x,["sim_nmFantasiaCleannmFantasiaOuroFuzz"]]=fuzz.ratio(data.loc[x,"nmFantasia_clean"], data.loc[x,'nomefantasia_base_ouro'])/100





# drop string variables



data.drop(
[
"Prestador_cdPrestadorGUID",
"nmPrestador",
"nmFantasia",
"dsEmail",
"dsTipoPrestador",
"nuCNPJ",
"nuCNES",
"nuCRM",
"nuInscricaoMunicipal",
"nuInscricaoEstadual",
"dsPrestadorUnidade",
"cdPrestadorNaOperadora",
"dsEspecialidade",
"longitude",
"latitude",
"nuTelefone",
"nuCEP",
"dsLogradouro",
"dsNumero",
"dsBairro",
"dsMunicipio",
"dsUf",
"dsComplemento",
"nmPrestador_clean",
"nmPrestador_levenstein",
"nmFantasia_clean",
"nuCNPJ_clean",
"nuCNES_clean",
"nuCRM_clean",
"nuInscricaoMunicipal_clean",
"nuInscricaoEstadual_clean",
"nuTelefone_clean",
"nuCEP_clean",
"dsLogradouro_clean",
"dsNumero_clean",
"dsBairro_clean",
"dsMunicipio_clean",
"dsUf_clean",
"dsComplemento_clean",
"dsEndereco",
"flagOrigem",
"cdTipoPrestador",
"key",
"idOperadora",
"idEmpresa",
"key_match",
"flagOrigem_match",
"idEmpresaOrigem_match",
"idOperadoraOrigem_match",
'cnpj_base_ouro',
"razaosocial_base_ouro",
"nomefantasia_base_ouro"
], axis=1, inplace=True)


data["flagHospital"]=data["flagHospital"].replace(np.nan, 0)
data["flagClinica"]=data["flagClinica"].replace(np.nan, 0)
data["flagHomeCare"]=data["flagHomeCare"].replace(np.nan, 0)
data["flagLaboratorio"]=data["flagLaboratorio"].replace(np.nan, 0)
data["flagCardiologia"]=data["flagCardiologia"].replace(np.nan, 0)
data["flagGinecologia"]=data["flagGinecologia"].replace(np.nan, 0)
data["flagPediatrico"]=data["flagPediatrico"].replace(np.nan, 0)
data["flagOrtopedia"]=data["flagOrtopedia"].replace(np.nan, 0)
data["flagOutros"]=data["flagOutros"].replace(np.nan, 0)




In [18]:
# Predição
predictions = rf_loaded_model.predict(data)

origi['match']=predictions

data.drop(
[
"sim_nmFantasiaCleanRazaosocialOuroJaro",
"sim_nmPrestadorCleanRazaosocialOuroJaro",
"sim_nmPrestadorCleannmFantasiaOuroFuzz",
"sim_nmFantasiaCleannmFantasiaOuroFuzz"

], axis=1, inplace=True)



In [ ]:


origi['match']=predictions
origi.to_excel(out_arq, index=True, header=True)
origi.head(50)
